# Spark DataFrame Basics

Spark DataFrames are the workhouse and main way of working with Spark and Python post Spark 2.0. DataFrames act as powerful versions of tables, with rows and columns, easily handling large datasets. The shift to DataFrames provides many advantages:
* A much simpler syntax
* Ability to use SQL directly in the dataframe
* Operations are automatically distributed across RDDs
    
If you've used R or even the pandas library with Python you are probably already familiar with the concept of DataFrames. Spark DataFrame expand on a lot of these concepts, allowing you to transfer that knowledge easily by understanding the simple syntax of Spark DataFrames. Remember that the main advantage to using Spark DataFrames vs those other programs is that Spark can handle data across many RDDs, huge data sets that would never fit on a single computer. That comes at a slight cost of some "peculiar" syntax choices, but after this course you will feel very comfortable with all those topics!

Let's get started!

## Creating a DataFrame

First we need to start a SparkSession:

Then start the SparkSession

In [3]:
from pyspark.sql import SparkSession

In [4]:
# May take a little while on a local computer
spark = SparkSession.builder.appName('Basics').getOrCreate()


You will first need to get the data from a file (or connect to a large distributed file like HDFS, we'll talk about this later once we move to larger datasets on AWS EC2).

In [5]:
df = spark.read.json('people.json')

**Showing the data**

In [6]:
# Note how data is missing!
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [7]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [8]:
df.columns

['age', 'name']

In [12]:
df.describe()

DataFrame[summary: string, age: string, name: string]

In [11]:
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



Some data types make it easier to infer schema (like tabular formats such as csv which we will show later). 

However you often have to set the schema yourself if you aren't dealing with a .read method that doesn't have inferSchema() built-in.

Spark has all the tools you need for this, it just requires a very specific structure:

In [15]:
from pyspark.sql.types import (StructField, StructType,
                               IntegerType, StringType)

Next we need to create the list of Structure fields
    * :param name: string, name of the field.
    * :param dataType: :class:`DataType` of the field.
    * :param nullable: boolean, whether the field can be null (None) or not.

In [14]:
data_schema = [StructField('age', IntegerType(), True), StructField('name', StringType(), True)]

In [16]:
final_struct = StructType(fields=data_schema)

In [17]:
df = spark.read.json('people.json', schema=final_struct)

In [18]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



### Grabing the data

In [22]:
type(df['age'])

pyspark.sql.column.Column

In [25]:
type(df.select('age'))

pyspark.sql.dataframe.DataFrame

In [24]:
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [26]:
# Return list of objects
df.head(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

In [27]:
# Multiple
df.select(['age', 'name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### Creating new columns

In [29]:
# Adding a new column with a simple copy multiplied by 2
df.withColumn('new_age', df['age']*2).show()

+----+-------+-------+
| age|   name|new_age|
+----+-------+-------+
|null|Michael|   null|
|  30|   Andy|     60|
|  19| Justin|     38|
+----+-------+-------+



In [30]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [31]:
# Simple rename
df.withColumnRenamed('age', 'newage').show()

+------+-------+
|newage|   name|
+------+-------+
|  null|Michael|
|    30|   Andy|
|    19| Justin|
+------+-------+



More complicated operations to create new columns

In [32]:
df.withColumn('add_one_age',df['age']+1).show()

+----+-------+-----------+
| age|   name|add_one_age|
+----+-------+-----------+
|null|Michael|       null|
|  30|   Andy|         31|
|  19| Justin|         20|
+----+-------+-----------+



In [33]:
df.withColumn('half_age',df['age']/2).show()

+----+-------+--------+
| age|   name|half_age|
+----+-------+--------+
|null|Michael|    null|
|  30|   Andy|    15.0|
|  19| Justin|     9.5|
+----+-------+--------+



In [34]:
df.withColumn('half_age',df['age']/2)

DataFrame[age: int, name: string, half_age: double]

### Using SQL

To use SQL queries directly with the dataframe, you will need to register it to a temporary view:

In [35]:
# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView('people')

In [36]:
sql_results = spark.sql('SELECT * FROM people')

In [37]:
sql_results

DataFrame[age: int, name: string]

In [38]:
sql_results.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [40]:
spark.sql('SELECT * FROM people WHERE age = 30').show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+

